In [2]:
import json

In [7]:
indicators = [
    # --- Program Indicators ---
    {"name": "PGM_500_IND", "description": "PVI_CDE LIKE 'FDMI%'", "category": "Program", "logic": {"op": "startswith", "field": "PVI_CDE", "value": "FDMI"}},
    {"name": "PGM_25_IND", "description": "ACQ_ID = 620987 AND TRAN_CDE = '1720'", "category": "Program", "logic": {"op": "and", "args": [{"op": "eq", "field": "ACQ_ID", "value": 620987}, {"op": "eq", "field": "TRAN_CDE", "value": "1720"}]}},
    {"name": "PGM_26_IND", "description": "ACQ_ID = 620987 AND TRAN_CDE = '1730'", "category": "Program", "logic": {"op": "and", "args": [{"op": "eq", "field": "ACQ_ID", "value": 620987}, {"op": "eq", "field": "TRAN_CDE", "value": "1730"}]}},

    # --- Transaction Type ---
    {"name": "TRAN_SALE_IND", "description": "TRAN_CDE = '1720'", "category": "Transaction", "logic": {"op": "eq", "field": "TRAN_CDE", "value": "1720"}},
    {"name": "TRAN_RT_IND", "description": "TRAN_CDE = '1730'", "category": "Transaction", "logic": {"op": "eq", "field": "TRAN_CDE", "value": "1730"}},
    {"name": "V232_IND", "description": "PROC_DT < '2024-04-12'", "category": "Transaction", "logic": {"op": "lt", "field": "PROC_DT", "value": "2024-04-12"}},
    {"name": "V241_IND", "description": "PROC_DT >= '2024-04-12'", "category": "Transaction", "logic": {"op": "gte", "field": "PROC_DT", "value": "2024-04-12"}},
    {"name": "FRAUD_ADJ_IND", "description": "0: REGULATED WITHOUT FRAUD ADJ; 1: REGULATED WITH FRAUD ADJ; 2: UNKNOWN", "category": "Transaction", "logic": {"op": "if", "args": [{"op": "eq", "field": "CRD_PRDCT_GRP_ID", "value": 9}, 1, {"op": "eq", "field": "CRD_PRDCT_GRP_ID", "value": 10}, 0, 2]}},

    # --- Card Presence ---
    {"name": "CARD_PRSNCE", "description": "CP/CNP based on SUBSTR(AUTH_POS_DATA_TXT,5,1)", "category": "Card", "logic": {"op": "if", "args": [{"op": "eq", "field": "AUTH_POS_DATA_TXT[4]", "value": "0"}, "CP", {"op": "eq", "field": "AUTH_POS_DATA_TXT[4]", "value": "1"}, "CNP", "CNP"]}},
    {"name": "STTL_CARD_PRSNCE", "description": "None if SUBSTR(AUTH_POS_DATA_TXT,5,1) is space or None, else value", "category": "Card", "logic": {"op": "if", "args": [{"op": "or", "args": [{"op": "eq", "field": "AUTH_POS_DATA_TXT[4]", "value": " "}, {"op": "is_null", "field": "AUTH_POS_DATA_TXT[4]"}]}, None, {"op": "get", "field": "AUTH_POS_DATA_TXT[4]"}]}},
    {"name": "AUTH_CP_CNP_IND", "description": "Complex nested logic for CP/CNP indicator", "category": "Card", "logic": {"op": "if", "args": [{"op": "eq", "field": "AUTH_CRD_PRSNCE", "value": "0"}, 0, {"op": "eq", "field": "AUTH_CRD_PRSNCE", "value": "1"}, 1, {"op": "if", "args": [{"op": "in", "field": "POS_ENT_MODE_REQ_CDE", "value": ["02","03","04","05","08","81","83","85","86","94"]}, 0, {"op": "in", "field": "POS_ENT_MODE_REQ_CDE", "value": ["07","90","91","92","93","82"]}, 1, {"op": "if", "args": [{"op": "in", "field": "AUTH_CRD_PRSNCE", "value": ["0"]}, 0, {"op": "in", "field": "AUTH_CRD_PRSNCE", "value": ["1"]}, 1, {"op": "if", "args": [{"op": "in", "field": "AUTH_CRD_PRSNCE", "value": ["0"]}, 0, {"op": "in", "field": "AUTH_CRD_PRSNCE", "value": ["1","2","3","4","5"]}, 1, {"op": "if", "args": [{"op": "in", "field": "STTL_STD_POS_ENT", "value": ["02","03","04","05","08","81","83","85","86","94"]}, 0, {"op": "in", "field": "STTL_STD_POS_ENT", "value": ["07","90","91","92","93","82"]}, 1, 0]}]}]}]}]}},
    {"name": "STTL_CP_CNP_IND", "description": "0 if SUBSTR(AUTH_POS_DATA_TXT,5,1) = '0', 1 if '1'", "category": "Card", "logic": {"op": "if", "args": [{"op": "eq", "field": "AUTH_POS_DATA_TXT[4]", "value": "0"}, 0, {"op": "eq", "field": "AUTH_POS_DATA_TXT[4]", "value": "1"}, 1, None]}},
    {"name": "STTL_AUTH_CNP_IND", "description": "STTL_CP_CNP_IND if not None, else AUTH_CP_CNP_IND", "category": "Card", "logic": {"op": "if", "args": [{"op": "not_null", "field": "STTL_CP_CNP_IND"}, {"op": "get", "field": "STTL_CP_CNP_IND"}, {"op": "get", "field": "AUTH_CP_CNP_IND"}]}},
    {"name": "VALID_CNP_IND", "description": "1 if STTL_AUTH_CNP_IND in (0,1), else 0", "category": "Card", "logic": {"op": "in", "field": "STTL_AUTH_CNP_IND", "value": [0, 1]}},

    # --- POS Entry Group ---
    {"name": "POS_ENT_GRP", "description": "CP/CNP/Unknown based on STTL_STD_POS_ENT", "category": "POS", "logic": {"op": "if", "args": [{"op": "in", "field": "STTL_STD_POS_ENT", "value": ["02","05","81","83","85","86"]}, "CP", {"op": "in", "field": "STTL_STD_POS_ENT", "value": ["01","07","10","82","91"]}, "CNP", "Unknown"]}},

    # --- TDCC Group ---
    {"name": "TDCC_GRP", "description": "CP/CNP/Invalid/Unknown based on STTL_TRCK_CODE", "category": "TDCC", "logic": {"op": "if", "args": [{"op": "in", "field": "STTL_TRCK_CODE", "value": ["02","03","20","30","05","50","06","80","81","82","84"]}, "CP", {"op": "in", "field": "STTL_TRCK_CODE", "value": ["70", "71", "72", "74", "90", "91", "99", "60", "61", "00"]}, "CNP", {"op": "in", "field": "STTL_TRCK_CODE", "value": ["04", "40"]}, "Invalid", "Unknown"]}},

    # --- Card Product Indicators ---
    {"name": "CS_CRD_IND", "description": "1 if REF_CRD_PRDCT_KEY not in (3,11,14,21,24,25,26,27), else 0", "category": "CardProduct", "logic": {"op": "not_in", "field": "REF_CRD_PRDCT_KEY", "value": [3,11,14,21,24,25,26,27]}},
    {"name": "CRD_PUBLIC_SRV_MAX_IND", "description": "1 if REF_CRD_PRDCT_KEY in (5,6,12), else 0", "category": "CardProduct", "logic": {"op": "in", "field": "REF_CRD_PRDCT_KEY", "value": [5,6,12]}},
    {"name": "CRD_COMM_B2B_IND", "description": "1 if REF_CRD_PRDCT_KEY in (27,28), else 0", "category": "CardProduct", "logic": {"op": "in", "field": "REF_CRD_PRDCT_KEY", "value": [27,28]}},
    {"name": "CRD_REPAY_IND", "description": "1 if REF_CRD_PRDCT_KEY in (5,6,12,16,22,23), else 0", "category": "CardProduct", "logic": {"op": "in", "field": "REF_CRD_PRDCT_KEY", "value": [5,6,12,16,22,23]}},

    # --- Time Window Indicators (use TIME_DAYS_COALESCE) ---
    {"name": "TML_WT_2_IND", "description": "1 if TIME_DAYS_COALESCE <= 2, else 0", "category": "TimeWindow", "logic": {"op": "lte", "field": "TIME_DAYS_COALESCE", "value": 2}},
    {"name": "TML_WT_3_IND", "description": "1 if TIME_DAYS_COALESCE <= 3, else 0", "category": "TimeWindow", "logic": {"op": "lte", "field": "TIME_DAYS_COALESCE", "value": 3}},
    {"name": "TML_WT_5_IND", "description": "1 if TIME_DAYS_COALESCE <= 5, else 0", "category": "TimeWindow", "logic": {"op": "lte", "field": "TIME_DAYS_COALESCE", "value": 5}},
    {"name": "TML_WT_7_IND", "description": "1 if TIME_DAYS_COALESCE <= 7, else 0", "category": "TimeWindow", "logic": {"op": "lte", "field": "TIME_DAYS_COALESCE", "value": 7}},
    {"name": "TML_WT_8_IND", "description": "1 if TIME_DAYS_COALESCE <= 8, else 0", "category": "TimeWindow", "logic": {"op": "lte", "field": "TIME_DAYS_COALESCE", "value": 8}},

    # --- Country Indicators ---
    {"name": "MCCA_MRCH_CNTRY_IND", "description": "1 if MRCH_CNTRY_CDE in (MEX, Caribbean, Central America), else 0", "category": "Country", "logic": {"op": "in", "field": "MRCH_CNTRY_CDE", "value": ["MEX","ABW","AIA","ANT","ATG","BES","BHS","BLM","BLZ","BMU","BRB","CUW","CYM","DMA","DOM","GRD","JAM","KNA","LCA","MAF","MSR","SXM","TCA","TTO","VCT","VGB","CRI","SLV","GTM","HND","NIC","PAN"]}},
    {"name": "SOUTH_AM_IND", "description": "1 if MRCH_CNTRY_CDE in (South America), else 0", "category": "Country", "logic": {"op": "in", "field": "MRCH_CNTRY_CDE", "value": ["PRY","BRA","ARG","PER","URY","CHL","BOL"]}},
    {"name": "US_MRCH_IND", "description": "1 if LENGTH(TRIM(MRCH_CNTRY_CDE)) = 2 OR MRCH_CNTRY_CDE in ('-','USA','GUM','ASM','MNP','UMI','VIR','FSM','MHL','PRI','PLW') OR MRCH_CNTRY_CDE is None, else 0", "category": "Country", "logic": {"op": "or", "args": [{"op": "eq", "field": "LEN_TRIM_MRCH_CNTRY_CDE", "value": 2}, {"op": "in", "field": "MRCH_CNTRY_CDE", "value": ["-","USA","GUM","ASM","MNP","UMI","VIR","FSM","MHL","PRI","PLW"]}, {"op": "is_null", "field": "MRCH_CNTRY_CDE"}]}},
    {"name": "US_ISS_IND", "description": "1 if ISS_CNTRY_CDE in ('-','USA','GUM','ASM','MNP','UMI','VIR','FSM','MHL','PRI','PLW'), else 0", "category": "Country", "logic": {"op": "in", "field": "ISS_CNTRY_CDE", "value": ["-","USA","GUM","ASM","MNP","UMI","VIR","FSM","MHL","PRI","PLW"]}},
    {"name": "CNTRY_ARG_IND", "description": "1 if MRCH_CNTRY_CDE in ('ARG'), else 0", "category": "Country", "logic": {"op": "in", "field": "MRCH_CNTRY_CDE", "value": ["ARG"]}},
    {"name": "CNTRY_BOL_IND", "description": "1 if MRCH_CNTRY_CDE in ('BOL'), else 0", "category": "Country", "logic": {"op": "in", "field": "MRCH_CNTRY_CDE", "value": ["BOL"]}},
    {"name": "CNTRY_BRA_IND", "description": "1 if MRCH_CNTRY_CDE in ('BRA'), else 0", "category": "Country", "logic": {"op": "in", "field": "MRCH_CNTRY_CDE", "value": ["BRA"]}},
    {"name": "CNTRY_CHL_IND", "description": "1 if MRCH_CNTRY_CDE in ('CHL'), else 0", "category": "Country", "logic": {"op": "in", "field": "MRCH_CNTRY_CDE", "value": ["CHL"]}},
    {"name": "CNTRY_PRY_IND", "description": "1 if MRCH_CNTRY_CDE in ('PRY'), else 0", "category": "Country", "logic": {"op": "in", "field": "MRCH_CNTRY_CDE", "value": ["PRY"]}},
    {"name": "CNTRY_PER_IND", "description": "1 if MRCH_CNTRY_CDE in ('PER'), else 0", "category": "Country", "logic": {"op": "in", "field": "MRCH_CNTRY_CDE", "value": ["PER"]}},
    {"name": "CNTRY_URY_IND", "description": "1 if MRCH_CNTRY_CDE in ('URY'), else 0", "category": "Country", "logic": {"op": "in", "field": "MRCH_CNTRY_CDE", "value": ["URY"]}},

    # --- Amount/Amount Tolerance ---
    {"name": "AMT_TLR_IND", "description": "Complex amount tolerance logic", "category": "Amount", "logic": {
        "op": "or",
        "args": [
            {"op": "and", "args": [{"op": "eq", "field": "TOT_TRAN_AMT", "value": 0}, {"op": "eq", "field": "TRAN_RSPNS_AMT", "value": 0}]},
            {"op": "and", "args": [{"op": "is_null", "field": "TOT_TRAN_AMT"}, {"op": "is_null", "field": "TRAN_RSPNS_AMT"}]},
            {"op": "and", "args": [{"op": "in", "field": "STTL_STD_POS_ENT", "value": ["05","83"]}, {"op": "in", "field": "STTL_AUTH_RSPNS_CDE", "value": ["Y1","Y3"]}]},
            {"op": "and", "args": [{"op": "in", "field": "STD_MCC_CDE", "value": ["4121","7230"]}, {"op": "between", "field": "AMT_RATIO", "low": 0.8, "high": 1.2}]},
            {"op": "eq", "field": "MCC_PASSENGER_IND", "value": 1},
            {"op": "between", "field": "STD_MCC_CDE", "low": "5815", "high": "5818"},
            {"op": "in", "field": "STD_MCC_CDE", "value": ["4111","4131","5812","5813","5814"]},
            {"op": "eq", "field": "MCC_HOTEL_CARRENT_IND", "value": 1},
            {"op": "eq", "field": "MCC_PETROLUEM_IND", "value": 1},
            {"op": "and", "args": [
                {"op": "eq", "field": "MCC_SUPERMARKET_IND", "value": 1},
                {"op": "between", "field": "AMT_RATIO", "low": 0.85, "high": 1.15}
            ]},
            {"op": "between", "field": "AMT_RATIO", "low": 0.9, "high": 1.1}
        ]
    }},
    {"name": "ECOMM_AMT_IND", "description": "1 if AMT_TLR_IND = 1 or PARTIAL_IND = 1, else 0", "category": "Amount", "logic": {"op": "or", "args": [{"op": "eq", "field": "AMT_TLR_IND", "value": 1}, {"op": "eq", "field": "PARTIAL_IND", "value": 1}]}},

    # --- MCC Indicators (first half) ---
    {"name": "MCC_RECURRING_IND", "description": "1 if STD_MCC_CDE in ('4814','4899'), else 0", "category": "MCC", "logic": {"op": "in", "field": "STD_MCC_CDE", "value": ["4814","4899"]}},
    {"name": "MCC_UTILITY_IND", "description": "1 if STD_MCC_CDE = '4900', else 0", "category": "MCC", "logic": {"op": "eq", "field": "STD_MCC_CDE", "value": "4900"}},
    {"name": "MCC_REALESTATE_IND", "description": "1 if STD_MCC_CDE = '6513', else 0", "category": "MCC", "logic": {"op": "eq", "field": "STD_MCC_CDE", "value": "6513"}},
    {"name": "MCC_REPAY_IND", "description": "1 if STD_MCC_CDE in ('6051','6012'), else 0", "category": "MCC", "logic": {"op": "in", "field": "STD_MCC_CDE", "value": ["6051","6012"]}},
    {"name": "MCC_INSURANCE_IND", "description": "1 if STD_MCC_CDE in ('5960','6300'), else 0", "category": "MCC", "logic": {"op": "in", "field": "STD_MCC_CDE", "value": ["5960","6300"]}},
    {"name": "MCC_SUPERMARKET_IND", "description": "1 if STD_MCC_CDE in ('5300','5411'), else 0", "category": "MCC", "logic": {"op": "in", "field": "STD_MCC_CDE", "value": ["5300","5411"]}},
    {"name": "MCC_EMERGMKT_IND", "description": "1 if STD_MCC_CDE in ('4899','5968','5983','6533','8211','8220','8299','8351'), else 0", "category": "MCC", "logic": {"op": "in", "field": "STD_MCC_CDE", "value": ["4899","5968","5983","6533","8211","8220","8299","8351"]}},
    {"name": "MCC_CHARITY_IND", "description": "1 if STD_MCC_CDE = '8398', else 0", "category": "MCC", "logic": {"op": "eq", "field": "STD_MCC_CDE", "value": "8398"}},
    {"name": "MCC_PUBLICSRV_IND", "description": "1 if STD_MCC_CDE in ('4784','9211','9222','9223','9311','9399','9405'), else 0", "category": "MCC", "logic": {"op": "in", "field": "STD_MCC_CDE", "value": ["4784","9211","9222","9223","9311","9399","9405"]}},
    {"name": "MCC_RESTUARANTS_IND", "description": "1 if STD_MCC_CDE in ('5812','5814'), else 0", "category": "MCC", "logic": {"op": "in", "field": "STD_MCC_CDE", "value": ["5812","5814"]}},
    {"name": "MCC_MICRO_TKT_IND", "description": "1 if STD_MCC_CDE in ('4111','4112','4131','4784','5499','5552','5994','7211','7216','7338','7523','7542','7832','7841') or MCC_RESTUARANTS_IND = 1, else 0", "category": "MCC", "logic": {"op": "or", "args": [{"op": "in", "field": "STD_MCC_CDE", "value": ["4111","4112","4131","4784","5499","5552","5994","7211","7216","7338","7523","7542","7832","7841"]}, {"op": "eq", "field": "MCC_RESTUARANTS_IND", "value": 1}]}},
    {"name": "MCC_EXPR_SRV_IND", "description": "1 if STD_MCC_CDE in ('4111','4112','4121','4131','4784','5499','5552','5994','7211','7216','7338','7523','7542','7832','7841') or MCC_RESTUARANTS_IND = 1, else 0", "category": "MCC", "logic": {"op": "or", "args": [{"op": "in", "field": "STD_MCC_CDE", "value": ["4111","4112","4121","4131","4784","5499","5552","5994","7211","7216","7338","7523","7542","7832","7841"]}, {"op": "eq", "field": "MCC_RESTUARANTS_IND", "value": 1}]}},
    {"name": "MCC_PETROLUEM_IND", "description": "1 if STD_MCC_CDE in ('5541','5542'), else 0", "category": "MCC", "logic": {"op": "in", "field": "STD_MCC_CDE", "value": ["5541","5542"]}},
    {"name": "MCC_HIGH_RISK_IND", "description": "1 if STD_MCC_CDE in ('5962','5966','5967'), else 0", "category": "MCC", "logic": {"op": "in", "field": "STD_MCC_CDE", "value": ["5962","5966","5967"]}},
    {"name": "MCC_PASSENGER_IND", "description": "1 if STD_MCC_CDE in ('4112','4411','4511') or (STD_MCC_CDE between '3000' and '3350'), else 0", "category": "MCC", "logic": {"op": "or", "args": [{"op": "in", "field": "STD_MCC_CDE", "value": ["4112","4411","4511"]}, {"op": "between", "field": "STD_MCC_CDE", "low": "3000", "high": "3350"}]}},
    {"name": "MCC_HOTEL_CARRENT_IND", "description": "1 if STD_MCC_CDE in ('7011','7012','7512','7513','7519') or between '3351'-'3441' or '3501'-'3999', else 0", "category": "MCC", "logic": {"op": "or", "args": [{"op": "in", "field": "STD_MCC_CDE", "value": ["7011","7012","7512","7513","7519"]}, {"op": "between", "field": "STD_MCC_CDE", "low": "3351", "high": "3441"}, {"op": "between", "field": "STD_MCC_CDE", "low": "3501", "high": "3999"}]}},
    {"name": "MCC_QUASI_IND", "description": "1 if STD_MCC_CDE in ('4829','6050','6051','6540'), else 0", "category": "MCC", "logic": {"op": "in", "field": "STD_MCC_CDE", "value": ["4829","6050","6051","6540"]}},
    {"name": "MCC_LARGE_TKT_IND", "description": "1 if STD_MCC_CDE in (long list), else 0", "category": "MCC", "logic": {"op": "in", "field": "STD_MCC_CDE", "value": ["2741","2791","2842","5013","5021","5039","5044","5045","5046","5047","5051","5065","5072","5074","5085","5094","5099","5199","5111","5122","5131","5137","5139","5169","5172","5192","5193","5198"]}},
    {"name": "MCC_CS_VOUCHER_IND", "description": "1 if MCC_HIGH_RISK_IND = 1 or STD_MCC_CDE in ('5960','5964','5965','5968','5969','5262'), else 0", "category": "MCC", "logic": {"op": "or", "args": [{"op": "eq", "field": "MCC_HIGH_RISK_IND", "value": 1}, {"op": "in", "field": "STD_MCC_CDE", "value": ["5960","5964","5965","5968","5969","5262"]}]}},

    # --- MCC Indicators (second half), POS, and all remaining indicators will be in Part 2 ---
]

In [8]:
indicators += [
    # --- MCC (continued) ---
    {
        "name": "MCC_RETAIL_IND",
        "description": "1 if all other MCC indicators are 0, else 0",
        "category": "MCC",
        "logic": {
            "op": "and",
            "args": [
                {"op": "eq", "field": "MCC_UTILITY_IND", "value": 0},
                {"op": "eq", "field": "MCC_REALESTATE_IND", "value": 0},
                {"op": "eq", "field": "MCC_INSURANCE_IND", "value": 0},
                {"op": "eq", "field": "MCC_SUPERMARKET_IND", "value": 0},
                {"op": "eq", "field": "MCC_EMERGMKT_IND", "value": 0},
                {"op": "eq", "field": "MCC_PUBLICSRV_IND", "value": 0},
                {"op": "eq", "field": "MCC_PETROLUEM_IND", "value": 0},
                {"op": "eq", "field": "MCC_RESTUARANTS_IND", "value": 0},
                {"op": "eq", "field": "MCC_HOTEL_CARRENT_IND", "value": 0},
                {"op": "eq", "field": "MCC_QUASI_IND", "value": 0},
                {"op": "eq", "field": "MCC_HIGH_RISK_IND", "value": 0},
                {"op": "eq", "field": "MCC_PASSENGER_IND", "value": 0}
            ]
        }
    },
    {
        "name": "MCC_CNP_ECOMM_IND",
        "description": "1 if all other MCC indicators are 0 (except MCC_RETAIL_IND), else 0",
        "category": "MCC",
        "logic": {
            "op": "and",
            "args": [
                {"op": "eq", "field": "MCC_UTILITY_IND", "value": 0},
                {"op": "eq", "field": "MCC_REALESTATE_IND", "value": 0},
                {"op": "eq", "field": "MCC_INSURANCE_IND", "value": 0},
                {"op": "eq", "field": "MCC_EMERGMKT_IND", "value": 0},
                {"op": "eq", "field": "MCC_HOTEL_CARRENT_IND", "value": 0},
                {"op": "eq", "field": "MCC_PUBLICSRV_IND", "value": 0},
                {"op": "eq", "field": "MCC_QUASI_IND", "value": 0},
                {"op": "eq", "field": "MCC_HIGH_RISK_IND", "value": 0},
                {"op": "eq", "field": "MCC_PASSENGER_IND", "value": 0}
            ]
        }
    },
    {
        "name": "MCC_COMM_ELECTRO_IND",
        "description": "1 if MCC_INSURANCE_IND = 0 and MCC_PASSENGER_IND = 0 and MCC_EMERGMKT_IND = 0 and MCC_PUBLICSRV_IND = 0 and MCC_HOTEL_CARRENT_IND = 0, else 0",
        "category": "MCC",
        "logic": {
            "op": "and",
            "args": [
                {"op": "eq", "field": "MCC_INSURANCE_IND", "value": 0},
                {"op": "eq", "field": "MCC_PASSENGER_IND", "value": 0},
                {"op": "eq", "field": "MCC_EMERGMKT_IND", "value": 0},
                {"op": "eq", "field": "MCC_PUBLICSRV_IND", "value": 0},
                {"op": "eq", "field": "MCC_HOTEL_CARRENT_IND", "value": 0}
            ]
        }
    },
    {
        "name": "CS_EMERG_MKT_IND",
        "description": "1 if MCC_EMERGMKT_IND = 1 and CRD_REPAY_IND = 0, else 0",
        "category": "MCC",
        "logic": {
            "op": "and",
            "args": [
                {"op": "eq", "field": "MCC_EMERGMKT_IND", "value": 1},
                {"op": "eq", "field": "CRD_REPAY_IND", "value": 0}
            ]
        }
    },
    {
        "name": "NEW_RETAIL_IND",
        "description": "1 if MCC_RETAIL_IND = 1 or CS_EMERG_MKT_IND = 1, else 0",
        "category": "MCC",
        "logic": {
            "op": "or",
            "args": [
                {"op": "eq", "field": "MCC_RETAIL_IND", "value": 1},
                {"op": "eq", "field": "CS_EMERG_MKT_IND", "value": 1}
            ]
        }
    },
    {
        "name": "NEW_CNP_IND",
        "description": "1 if MCC_CNP_ECOMM_IND = 1 or CS_EMERG_MKT_IND = 1, else 0",
        "category": "MCC",
        "logic": {
            "op": "or",
            "args": [
                {"op": "eq", "field": "MCC_CNP_ECOMM_IND", "value": 1},
                {"op": "eq", "field": "CS_EMERG_MKT_IND", "value": 1}
            ]
        }
    },

    # --- POS Indicators ---
    {
        "name": "POS_SWIPE_IND_STTL",
        "description": "1 if STTL_STD_POS_ENT in ('02','05','81','83','85','86'), else 0",
        "category": "POS",
        "logic": {
            "op": "in",
            "field": "STTL_STD_POS_ENT",
            "value": ["02","05","81","83","85","86"]
        }
    },
    {
        "name": "POS_CNP_IND_STTL",
        "description": "1 if STTL_STD_POS_ENT in ('01','10','91'), else 0",
        "category": "POS",
        "logic": {
            "op": "in",
            "field": "STTL_STD_POS_ENT",
            "value": ["01","10","91"]
        }
    },
    {
        "name": "POS_ECOM_IND_STTL",
        "description": "1 if STTL_STD_POS_ENT in ('07','10','82'), else 0",
        "category": "POS",
        "logic": {
            "op": "in",
            "field": "STTL_STD_POS_ENT",
            "value": ["07","10","82"]
        }
    },
    {
        "name": "POS_ECOM_SECURED_IND_STTL",
        "description": "1 if STTL_STD_POS_ENT in ('01','07','10','82'), else 0",
        "category": "POS",
        "logic": {
            "op": "in",
            "field": "STTL_STD_POS_ENT",
            "value": ["01","07","10","82"]
        }
    },
    {
        "name": "POS_ENT_KEY_STTL",
        "description": "1 if STTL_STD_POS_ENT = '01', else 0",
        "category": "POS",
        "logic": {
            "op": "eq",
            "field": "STTL_STD_POS_ENT",
            "value": "01"
        }
    },
    {
        "name": "POS_REPAY_IND_STTL",
        "description": "1 if STTL_STD_POS_ENT in ('01','07','10','82','91'), else 0",
        "category": "POS",
        "logic": {
            "op": "in",
            "field": "STTL_STD_POS_ENT",
            "value": ["01","07","10","82","91"]
        }
    },

    # --- Other/Complex Indicators ---
    {
        "name": "PROC_RB_IND_STTL",
        "description": "1 if PROC_CDE LIKE '13%' or '14%' or '15%' or '16%', else 0",
        "category": "Other",
        "logic": {
            "op": "or",
            "args": [
                {"op": "startswith", "field": "PROC_CDE", "value": "13"},
                {"op": "startswith", "field": "PROC_CDE", "value": "14"},
                {"op": "startswith", "field": "PROC_CDE", "value": "15"},
                {"op": "startswith", "field": "PROC_CDE", "value": "16"}
            ]
        }
    },
    {
        "name": "OFFLINE_CHIP_APR",
        "description": "1 if STTL_STD_POS_ENT in ('05','83','86') AND STTL_AUTH_RSPNS_CDE in ('Y1','Y3'), else 0",
        "category": "Other",
        "logic": {
            "op": "and",
            "args": [
                {"op": "in", "field": "STTL_STD_POS_ENT", "value": ["05","83","86"]},
                {"op": "in", "field": "STTL_AUTH_RSPNS_CDE", "value": ["Y1","Y3"]}
            ]
        }
    },
    {
        "name": "AUTH_RSPN_IND",
        "description": "1 if (STTL_STD_POS_ENT in ('05','83','86') AND STTL_AUTH_RSPNS_CDE in ('Y1','Y3')) OR STTL_AUTH_RSPNS_CDE in ('00','10','11'), else 0",
        "category": "Other",
        "logic": {
            "op": "or",
            "args": [
                {
                    "op": "and",
                    "args": [
                        {"op": "in", "field": "STTL_STD_POS_ENT", "value": ["05","83","86"]},
                        {"op": "in", "field": "STTL_AUTH_RSPNS_CDE", "value": ["Y1","Y3"]}
                    ]
                },
                {"op": "in", "field": "STTL_AUTH_RSPNS_CDE", "value": ["00","10","11"]}
            ]
        }
    },
    {
        "name": "TRCK_COND_IND_STTL",
        "description": "Complex TDCC logic",
        "category": "TDCC",
        "logic": {
            "op": "if",
            "args": [
                {
                    "op": "and",
                    "args": [
                        {"op": "in", "field": "STTL_STD_POS_ENT", "value": ["05","83","86"]},
                        {"op": "in", "field": "STTL_AUTH_RSPNS_CDE", "value": ["Y1","Y3"]}
                    ]
                }, 1,
                {
                    "op": "and",
                    "args": [
                        {"op": "in", "field": "STTL_STD_POS_ENT", "value": ["02","81","85","86"]},
                        {"op": "not_in", "field": "STTL_TRCK_CODE", "value": ["02","03","05","06","20","30","50","80","81","82","84"]}
                    ]
                }, 0,
                {
                    "op": "and",
                    "args": [
                        {"op": "in", "field": "STTL_STD_POS_ENT", "value": ["05","83","86"]},
                        {"op": "in", "field": "STTL_AUTH_RSPNS_CDE", "value": ["00","10","11"]},
                        {"op": "not_in", "field": "STTL_TRCK_CODE", "value": ["02","03","05","06","20","30","50","80","81","82","84"]}
                    ]
                }, 0,
                {
                    "op": "and",
                    "args": [
                        {"op": "in", "field": "STTL_STD_POS_ENT", "value": ["01","07","10","82"]},
                        {"op": "eq", "field": "SECURED_ECOMM_IND", "value": 1},
                        {"op": "not_in", "field": "STTL_TRCK_CODE", "value": ["60","61"]}
                    ]
                }, 0,
                1
            ]
        }
    },
    {
        "name": "TRCK_COND_RP_IND_STTL",
        "description": "TDCC for Consumer Debit Repayment",
        "category": "TDCC",
        "logic": {
            "op": "if",
            "args": [
                {
                    "op": "and",
                    "args": [
                        {"op": "in", "field": "STTL_STD_POS_ENT", "value": ["05","83","86"]},
                        {"op": "in", "field": "STTL_AUTH_RSPNS_CDE", "value": ["Y1","Y3"]}
                    ]
                }, 1,
                {
                    "op": "and",
                    "args": [
                        {"op": "in", "field": "STTL_STD_POS_ENT", "value": ["01","07","10","82","91"]},
                        {"op": "not_in", "field": "STTL_TRCK_CODE", "value": ["00","60","70","71","72","74"]}
                    ]
                }, 0,
                1
            ]
        }
    },
    {
        "name": "AVS_IND_STTL",
        "description": "AVS indicator",
        "category": "Other",
        "logic": {
            "op": "or",
            "args": [
                {
                    "op": "and",
                    "args": [
                        {"op": "not_null", "field": "AVS_PSTL_CDE"},
                        {"op": "neq", "field": "AVS_PSTL_CDE", "value": "-"}
                    ]
                },
                {
                    "op": "or",
                    "args": [
                        {"op": "startswith", "field": "PROC_CDE", "value": "13"},
                        {"op": "startswith", "field": "PROC_CDE", "value": "14"},
                        {"op": "startswith", "field": "PROC_CDE", "value": "15"},
                        {"op": "startswith", "field": "PROC_CDE", "value": "16"}
                    ]
                },
                {"op": "eq", "field": "STTL_STD_POS_ENT", "value": "82"}
            ]
        }
    },
    {
        "name": "NRID_IND",
        "description": "NRID indicator",
        "category": "Other",
        "logic": {
            "op": "if",
            "args": [
                {
                    "op": "and",
                    "args": [
                        {"op": "in", "field": "STTL_AUTH_RSPNS_CDE", "value": ["Y1","Y3"]},
                        {"op": "in", "field": "STTL_STD_POS_ENT", "value": ["05","83","86"]}
                    ]
                }, 1,
                {"op": "neq", "field": "LENGTH_TRIM_NWK_REF_ID", "value": 15}, 0,
                {"op": "eq", "field": "LENGTH_REPLACE_NWK_REF_ID", "value": 0}, 0,
                1
            ]
        }
    },
    {
        "name": "AUTH_APRV_IND",
        "description": "1 if STTL_AUTH_RSPNS_CDE in ('00','10','11'), else 0",
        "category": "Other",
        "logic": {
            "op": "in",
            "field": "STTL_AUTH_RSPNS_CDE",
            "value": ["00","10","11"]
        }
    },
    {
        "name": "EMV_SDR_IND_STTL",
        "description": "EMV SDR indicator",
        "category": "Other",
        "logic": {
            "op": "or",
            "args": [
                {
                    "op": "and",
                    "args": [
                        {"op": "in", "field": "STTL_STD_POS_ENT", "value": ["05","83","86"]},
                        {"op": "eq", "field": "CHIP_CRD_SDR_IND", "value": "Y"}
                    ]
                },
                {
                    "op": "not_in", "field": "STTL_STD_POS_ENT", "value": ["05","83","86"]
                }
            ]
        }
    },
    # ... (Add any remaining indicators as needed) ...
]

In [9]:
import json
from collections import defaultdict

# Step 1: Group by category
category_dict = defaultdict(list)
for indicator in indicators:
    category = indicator['category']
    category_dict[category].append(indicator)

# Step 2: Write each category to a JSON file
for category, items in category_dict.items():
    # Clean up the filename (optional: make lowercase, remove spaces, etc.)
    filename = f"{category}.json"
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(items, f, indent=2, ensure_ascii=False)

print("JSON files created for each category.")

JSON files created for each category.
